Importando bibliotecas

In [126]:
import pickle
import pathlib

import numpy as np
import pandas as pd

#### Carregando base de dados com a feature engineering 

In [127]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

c:\Users\Alan Matheus\Downloads\ames\data


In [128]:
clean_data_path = DATA_DIR / 'processed' / 'ames_minha1.pkl'

In [129]:
with open('../data/processed/ames_minha1.pkl', 'rb') as file:
    data = pickle.load(file)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2724 entries, 0 to 2929
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   MS.SubClass      2724 non-null   category
 1   MS.Zoning        2724 non-null   category
 2   Lot.Frontage     2724 non-null   float64 
 3   Lot.Area         2724 non-null   float64 
 4   Lot.Shape        2724 non-null   category
 5   Land.Contour     2724 non-null   category
 6   Lot.Config       2724 non-null   category
 7   Land.Slope       2724 non-null   category
 8   Neighborhood     2724 non-null   category
 9   Bldg.Type        2724 non-null   category
 10  House.Style      2724 non-null   category
 11  Overall.Qual     2724 non-null   category
 12  Overall.Cond     2724 non-null   category
 13  Year.Built       2724 non-null   float64 
 14  Roof.Style       2724 non-null   category
 15  Mas.Vnr.Type     2724 non-null   category
 16  Mas.Vnr.Area     2724 non-null   float64 
 17  

In [130]:
model_data = data.copy()

## Transformando todas as colunas em numéricas

In [131]:
categorical_columns = []
ordinal_columns = []
for col in model_data.select_dtypes('category').columns:
    if model_data[col].cat.ordered:
        ordinal_columns.append(col)
    else:
        categorical_columns.append(col)

### Colunas ordinais


In [132]:
for col in ordinal_columns:
    codes, _ = pd.factorize(data[col], sort=True)
    model_data[col] = codes

### Colunas categóricas

In [133]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
AsbShng,False,False,False,False,False
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False


In [134]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False
Other,False,False,False,False,False


#### Transformação feita com one-hot enconding, retirando a primeira coluna

In [135]:
model_data = pd.get_dummies(model_data, drop_first=True)

In [160]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2724 entries, 0 to 2929
Columns: 173 entries, Lot.Frontage to Exterior_Other
dtypes: bool(117), float64(42), int64(14)
memory usage: 1.5 MB


**Quais colunas foram criadas a partir de cada uma das colunas categóricas**

In [136]:
for cat in categorical_columns:
    dummies = []
    for col in model_data.columns:
        if col.startswith(cat + "_"):
            dummies.append(f'"{col}"')
    dummies_str = ', '.join(dummies)
    print(f'From column "{cat}" we made {dummies_str}\n')

From column "MS.SubClass" we made "MS.SubClass_30", "MS.SubClass_50", "MS.SubClass_60", "MS.SubClass_70", "MS.SubClass_80", "MS.SubClass_85", "MS.SubClass_90", "MS.SubClass_120", "MS.SubClass_160", "MS.SubClass_190", "MS.SubClass_Other"

From column "MS.Zoning" we made "MS.Zoning_RH", "MS.Zoning_RL", "MS.Zoning_RM"

From column "Land.Contour" we made "Land.Contour_HLS", "Land.Contour_Low", "Land.Contour_Lvl"

From column "Lot.Config" we made "Lot.Config_CulDSac", "Lot.Config_FR2", "Lot.Config_FR3", "Lot.Config_Inside"

From column "Neighborhood" we made "Neighborhood_BrDale", "Neighborhood_BrkSide", "Neighborhood_ClearCr", "Neighborhood_CollgCr", "Neighborhood_Crawfor", "Neighborhood_Edwards", "Neighborhood_Gilbert", "Neighborhood_IDOTRR", "Neighborhood_MeadowV", "Neighborhood_Mitchel", "Neighborhood_NAmes", "Neighborhood_NPkVill", "Neighborhood_NWAmes", "Neighborhood_NoRidge", "Neighborhood_NridgHt", "Neighborhood_OldTown", "Neighborhood_SWISU", "Neighborhood_Sawyer", "Neighborhood_Sa

## Separação dos dados de trenainamento e teste

In [137]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [138]:
X.values, y.values

(array([[141.0, 31770.0, 1, ..., False, False, False],
        [80.0, 11622.0, 0, ..., False, False, False],
        [81.0, 14267.0, 1, ..., True, False, False],
        ...,
        [68.0, 8885.0, 1, ..., False, False, False],
        [77.0, 10010.0, 0, ..., False, False, False],
        [74.0, 9627.0, 0, ..., False, False, False]], dtype=object),
 array([5.33243846, 5.0211893 , 5.23552845, ..., 5.1172713 , 5.23044892,
        5.27415785]))

In [139]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42  # Any number here, really.
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

## Treinando modelos para encontrar os melhores hiperparâmetros

In [140]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

### Rigde

In [141]:
# gs_ridge = GridSearchCV( estimator = Ridge(random_state=RANDOM_SEED),
#                             param_grid = {
#                                 'alpha': np.logspace(-3, 3, 7),
#                                 'fit_intercept': [True, False],
#                                 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
#                                 'tol': [1e-3, 1e-4, 1e-5],

#                                           },
#                             cv = 5,
#                             n_jobs = -1)

# gs_ridge.fit(Xtrain, ytrain)
# print(gs_ridge.best_params_)

{'alpha': 10.0, 'fit_intercept': True, 'solver': 'svd', 'tol': 0.001}

### Lasso

In [142]:
# gs_lasso = GridSearchCV( estimator = Lasso(random_state=RANDOM_SEED),
#                             param_grid = {
#                                     'alpha': [0.00001, 0.0001, 0.001, 0.01],
#                                     'fit_intercept': [True, False],
#                                     'max_iter': [250,500, 1000, 5000, 10000,100000]
#                                           },
#                             cv = 5,
#                             n_jobs = -1)

# gs_lasso.fit(Xtrain, ytrain)
# print(gs_lasso.best_params_)

{'alpha': 0.0001, 'fit_intercept': True, 'max_iter': 100000}

### Gradient Boosting

In [143]:
# gs_gradient_booting = GridSearchCV( estimator = GradientBoostingRegressor(random_state=RANDOM_SEED),
#                             param_grid = {'n_estimators': [500, 1000, 2000],
#                                           'max_depth': [1, 2, 3, 4],
#                                           'learning_rate': [0.1, 0.01, 0.001],
#                                           },
#                             cv = 5,
#                             n_jobs = -1)

# gs_gradient_booting.fit(Xtrain, ytrain)
# print(gs_gradient_booting.best_params_)

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 2000}

### Random Forest Regressor

In [144]:

# gs_random = GridSearchCV( estimator = RandomForestRegressor(random_state=RANDOM_SEED),
#                             param_grid = {
#                                 'n_estimators': [500, 1000, 2000],
#                                 'max_depth': [1, 2, 3, 4],
#                                           },
#                             cv = 5,
#                             n_jobs = -1)

# gs_random.fit(Xtrain, ytrain)
# print(gs_random.best_params_)

{'max_depth': 4, 'n_estimators': 2000}

## Comparando os modelos com validação cruzada

In [145]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(alpha= 10.0, fit_intercept= True, solver= 'svd', tol= 0.001, random_state=RANDOM_SEED),
    "Lasso": Lasso(alpha=0.00001, fit_intercept=True, max_iter=100000, random_state=RANDOM_SEED),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=2000, max_depth=4, learning_rate=0.1, random_state=RANDOM_SEED),
    "Random Forest": RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=2000, max_depth=4)
}

def erro_percentual(score):
    return 100 * (10**np.sqrt(-score) - 1)

model_scores = {}
pipelines = []
for model_name, model in models.items():
    # Criando o pipeline com normalização e o modelo
    pipeline = Pipeline([
        ('scaler', StandardScaler()), 
        ('model', model)
    ])
    pipelines.append(pipeline)

    # Validação Cruzada
    scores = cross_val_score(pipeline, Xtrain, ytrain, 
                                    scoring="neg_mean_squared_error", cv=8, n_jobs=-1)
    
    model_scores[model_name] = scores
    

    # Calculando o erro percentual
    error_percent_min = erro_percentual(scores.max())
    error_percent_max = erro_percentual(scores.min())
    error_percent_mean = erro_percentual(scores.mean())

    
    # Imprimindo o resultado
    print(f'Model: {model_name} - Error: {error_percent_min:.2f}%, {error_percent_max:.2f}%  => Mean Error {error_percent_mean:.2f}%')


C:\Users\Alan Matheus\AppData\Local\Temp\ipykernel_29588\3951425293.py:10: RuntimeWarning: overflow encountered in scalar power
  return 100 * (10**np.sqrt(-score) - 1)


Model: Linear Regression - Error: 8.51%, inf%  => Mean Error inf%
Model: Ridge - Error: 8.45%, 14.86%  => Mean Error 11.21%
Model: Lasso - Error: 8.50%, 14.87%  => Mean Error 11.29%
Model: Gradient Boosting - Error: 9.16%, 15.46%  => Mean Error 11.96%
Model: Random Forest - Error: 13.18%, 20.33%  => Mean Error 17.01%


In [153]:
import joblib

model_names = ["linear_regression", "ridge", "lasso", "gradient_boosting", "random_forest"]

for model_name, pipeline in zip(model_names, pipelines):
    pipeline.fit(Xtrain, ytrain)
    joblib.dump(pipeline, f"{model_name}.joblib")

A partir desses resultados podemos chegar em algumas conclusões:

* A taxa de erro do linear regression em certo caso pode tender ao infinito, o que torna arriscado o uso desse modelo;
* A taxa de erro do Random Forest é consideravelmente maior que os demais;
* A taxa de erros entre os outros três modelos está dentro de uma faixa pequena de diferença, ou seja, é necessário realizar mais testes comparativos para selecionar o modelo final.

## Teste de hipótese

**Pergunta:**  "Será que o desempenho médio (ou seja, $\overline{\text{RMSE}}$) de algum dos modelos é superior ao desempenho médio dos demais modelos de regressão?"

Para os seguintes teste, vamos adotar um nível de significância $\alpha$ de $0.01$ ($1 \%$) e nomear as médias populacionais do RSME de cada um dos modelos como $\mu_{\text{R}}$ (Ridge), $\mu_{\text{L}}$ (Lasso) e $\mu_{\text{GB}}$ (Gradient Boosting)

### Teste U de Mann-Whitney



In [147]:
from scipy.stats import mannwhitneyu
def rmse_score(score): return np.sqrt(-score)

#### Ridge vs Lasso

*Teste #1*: Comparar *Ridge* com *Lasso*.

*Hipótese nula* $H_0$: $\mu_{\text{R}} = \mu_{\text{L}}$

*Hipótese alternativa* $H_1$: $\mu_{\text{R}} \neq \mu_{\text{L}}$

In [148]:
U, pvalue = mannwhitneyu(rmse_score(model_scores['Ridge']), rmse_score(model_scores['Lasso']))
print("p-value: {0}".format(pvalue))


p-value: 0.5053613053613053


#### Ridge vs Gradient Boosting


*Teste #2*: Comparar *Ridge* com *Gradient Boosting*.

*Hipótese nula* $H_0$: $\mu_{\text{R}} = \mu_{\text{GB}}$

*Hipótese alternativa* $H_1$: $\mu_{\text{R}} \neq \mu_{\text{GB}}$

In [149]:
U, pvalue = mannwhitneyu(rmse_score(model_scores['Ridge']), rmse_score(model_scores['Gradient Boosting']))
print("p-value: {0}".format(pvalue))

p-value: 0.1605283605283605


#### Lasso vs Gradient Boosting


*Teste #3*: Comparar *Lasso* com *Gradient Boosting*.

*Hipótese nula* $H_0$: $\mu_{\text{L}} = \mu_{\text{GB}}$

*Hipótese alternativa* $H_1$: $\mu_{\text{L}} \neq \mu_{\text{GB}}$

In [150]:
U, pvalue = mannwhitneyu(rmse_score(model_scores['Lasso']), rmse_score(model_scores['Gradient Boosting']))
print("p-value: {0}".format(pvalue))

p-value: 0.1605283605283605


Como nenhum dos p-valores dos testes realizados teve um valor maior do que $\alpha=0.01$, então não podemos refutar a hipótese nula de que seus desempenhos são iguais. Desse modo, pode-se concluir que não há um nível de diferença entre os modelos grande o suficiente para que seja aparente no teste de hipótese realizado

## Avaliação final de desempenho

Após escolher o modelo final, devemos treiná-lo com o conjunto total de treino e calcular a taxa de erro com o conjunto de teste

In [151]:
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.fit_transform(Xtest)

final_model = Ridge(alpha= 10.0, fit_intercept= True, solver= 'svd', tol= 0.001, random_state=RANDOM_SEED)


# Ajustando o modelo
final_model.fit(Xtrain, ytrain)

# Fazendo previsões
ypred = final_model.predict(Xtest)

# Calculando o RMSE
RMSE = np.sqrt(mean_squared_error(ytest, ypred))

# Calculando o erro percentual
error_percent = 100 * (10**RMSE - 1)

# Imprimindo o resultado
print(f'Average error: {error_percent:.2f}%')

Average error: 9.98%


## Importância das features

In [152]:
attributes = list(X.columns)
feature_importances = final_model.coef_
sorted(zip(feature_importances, attributes), key=lambda x: abs(x[0]), reverse=True)[:10]

[(0.02785072220901855, 'Overall.Qual'),
 (0.022692824154205135, 'TotalSFLog'),
 (0.01970348916196259, 'Overall.Cond'),
 (0.018721398803690868, 'Gr.Liv.Area'),
 (0.012624178205388296, 'Sale.Condition_Normal'),
 (-0.01239247813396666, 'House.Age'),
 (0.012333385214885745, 'Year.Built'),
 (0.012089258736372815, 'Exterior_VinylSd'),
 (0.011261495275936558, 'X1st.Flr.SF'),
 (0.011161712895193264, 'X2nd.Flr.SF')]

## Treinamento final

Treinamento do modelo com o conjunto completo de dados

## Conclusão

*Quais as consequências do desempenho do modelo final para a aplicação de negócios?*

A taxa de erro de 9.98% encontrada através do conjunto de teste representa uma estimativa do desempenho do modelo em uma aplicação de negócios. Ou seja, caso esse modelo for utilizado em um sistema de produção 